# CityBikes

Send a request to CityBikes for the city of your choice. 

In [1]:
import pandas as pd
import requests
import json

In [4]:
#first api call to attain the 'href' for Los Angeles, CA
url_city = 'http://api.citybik.es/v2/networks'
result_city = requests.get(url_city)
cities = result_city.json()
for city in cities['networks']:
    if city['location']['city'] == 'Los Angeles, CA':
        print(city['href'])

/v2/networks/metro-bike-share


In [20]:
# 2nd api call to attain a list of each bike station
url_stations = 'http://api.citybik.es/v2/networks/metro-bike-share'
result_stations = requests.get(url_stations)
stations = result_stations.json()

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [39]:
station_info = stations['network']['stations']

{'empty_slots': 9,
 'extra': {'address': '699 S Imperial St',
  'last_updated': 1693492434,
  'rental_uris': {'android': 'https://www.bcycle.com/applink?system_id=bcycle_lametro&station_id=bcycle_lametro_3037&platform=android',
   'ios': 'https://www.bcycle.com/applink?system_id=bcycle_lametro&station_id=bcycle_lametro_3037&platform=iOS'},
  'renting': 1,
  'returning': 1,
  'uid': 'bcycle_lametro_3037'},
 'free_bikes': 11,
 'id': 'b6faafb631c1e75fe42dcf0183d5cdd9',
 'latitude': 34.0348,
 'longitude': -118.23128,
 'name': 'Imperial & 7th',
 'timestamp': '2023-08-31T14:33:54.946000Z'}

Put your parsed results into a DataFrame.

In [37]:
# create a dataframe with info for each station
bikes_df = pd.json_normalize(station_info)
bikes_df = bikes_df.rename(columns={'name':'station_address', 'latitude':'bike_station_lat','longitude':'bike_station_long'})
bikes_df = bikes_df[['station_address','bike_station_lat','bike_station_long','free_bikes']]
bikes_df.to_csv('bikes_df', index=False)